# Load NSA

Get an NSA catalog which includes both iauname and petrotheta/elpetro_absmag_r - v1_0_1

TODO replace this with the "key columns" version I already saved


In [2]:
from astropy.table import Table

In [4]:
# nsa = pd.read_csv('/media/walml/beta1/galaxy_zoo/decals/catalogs/nsa_v1_0_1_cached.csv')  # all nsa, but not crossmatched
nsa = Table.read('/Volumes/beta/galaxy_zoo/decals/catalogs/raw/nsa_v1_0_1.fits')  # ?
# nsa = pd.read_csv('/media/walml/beta1/galaxy_zoo/decals/catalogs/decals_dr5_uploadable_master_catalog_nov_2019.csv')  # crossmatched but excludes dr1/2
# nsa = pd.read_csv('/home/walml/repos/zoobot_private/data/decals/decals_master_catalog.csv')  # similarly as above


In [ ]:
nsa['ELPETRO_ABSMAG_R'] = nsa['ELPETRO_ABSMAG'][:, 4]
nsa['SERSIC_NMGY_R'] = nsa['SERSIC_NMGY'][:, 4]

In [ ]:
cols_to_keep = ['iauname', 'ra', 'dec', 'petro_theta', 'petro_th50', 'petro_th90', 'elpetro_absmag_r', 'sersic_nmgy_r']
for col in cols_to_keep:
    nsa[col] = nsa[col.upper()]

In [ ]:
nsa = nsa[cols_to_keep].to_pandas()

# Apply NSA cuts

Check petrotheta cut and apply r cuts matching dr8 and bulk of gz decals - 14 to 17.77


In [ ]:
# print(nsa['petro_theta'].min())  # cut not yet applied
good_radius = nsa['petro_theta'] > 3.
print(good_radius.mean())

In [ ]:
# http://www.sdss3.org/dr8/algorithms/magnitudes.php
nsa['mag_r'] = 22.5 - 2.5 * np.log10(nsa['sersic_nmgy_r'])

bad_mag = np.isnan(nsa['mag_r']) | np.isinf(nsa['mag_r'])

plt.hist(nsa[~bad_mag]['mag_r'], bins=np.linspace(0., 30.))
plt.axvline(17.77, color='r')
plt.axvline(14., color='r')
plt.xlabel('mag (r)')
plt.ylabel('galaxies')

In [ ]:
np.nan < 17.77, np.nan > 14  # nan will be removed by this cut as well, no need to exclude separately

In [ ]:
good_mag = (nsa['mag_r'] < 17.77) & (nsa['mag_r'] > 14.)
print(good_mag.mean())
print((good_mag & good_radius).mean())

# Drop galaxies with missing pixels

Get joint catalog and drop images with too many missing pixels - count to catalog so can change later, then to 5% or 7%



In [ ]:
# need to merge with to_upload in order to get galaxies with good downloads
# radius cut was already applied, but no mag cut. No previous galaxies were removed
# galaxies with missing pixels already removed
dr5_catalog = Table.read('/Volumes/beta/galaxy_zoo/decals/catalogs/dr5_nsa_v1_0_0_to_upload.fits')
print(len(dr5_catalog))
dr5_catalog['fits_ready'].mean(), dr5_catalog['png_ready'].mean()

In [ ]:
dr5_catalog.columns

In [ ]:
# just checking they merge correctly
test_df = pd.merge(nsa, dr5_catalog[['iauname', 'png_loc']].to_pandas(), on='iauname', how='inner')
print(len(nsa), len(dr5_catalog), len(test_df))

In [ ]:
safe_df = pd.merge(nsa[good_mag & good_radius], dr5_catalog[['iauname', 'png_loc']].to_pandas(), on='iauname', how='inner')
print(len(nsa), len(dr5_catalog), len(safe_df))

In [ ]:
safe_df['iauname'] = safe_df['iauname'].str.decode('utf8')
safe_df['png_loc'] = safe_df['png_loc'].str.decode('utf8')
safe_df['png_loc'] = safe_df['png_loc'].str.replace('/Volumes/alpha/decals', '/Volumes/beta/decals')


In [ ]:
safe_df['png_loc']

In [ ]:
assert os.path.isfile(safe_df['png_loc'][0])

In [ ]:
safe_df.head()

This catalog is the NSAxDR5 galaxies that could have been uploaded to GZ DECaLS, with:
- mag cuts
- radius cuts
- missing pixel cuts

In [ ]:
safe_df.to_parquet('/media/walml/beta1/cnn_features/sub_dr5_catalog_mag_radius_and_pixel_cuts.parquet', index=False)

In [ ]:

# check for duplicates in both dr5 and dr8 using ra/dec and crossmatch

# then create features and filename catalog in style of mid part of dr8 script

# then concat with dr8 table
# then apply rest of dr8  script